# Calling API to retrieve JSONs for exchange rate on historical dates

In [1]:
import os
import pandas as pd
from zipfile import ZipFile
import datetime
from datetime import datetime as dt
from datetime import date

import requests
import json

In [2]:
# setting up csv to be able to retrieve information

zf = ZipFile('archive.zip', 'r')
zf.extract('wfp_food_prices_database.csv')
zf.close()

df = pd.read_csv('wfp_food_prices_database.csv')
os.remove('wfp_food_prices_database.csv')
df.head(5)


c:\Users\Lilly\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
0,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,1,2014,50.0,NaN
1,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,2,2014,50.0,NaN
2,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,3,2014,50.0,NaN
3,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,4,2014,50.0,NaN
4,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,5,2014,50.0,NaN


In [3]:
# Create Datetime object for each row
# Isolate just the month and date of each row
tempDayMonthDF = df.loc[:, 'mp_month':'mp_year']

# Define a temp method for creating a datetime object from a row of our dataframe
def _create_datetime(row):
    return datetime.fromisoformat(f"{row[1]}-{row[0]:02d}-01")

# Add new datetime row to the dataframe
tempDayMonthDF = tempDayMonthDF.apply(_create_datetime, axis=1)
df.insert(0, 'Date', tempDayMonthDF)
df.head()

,Date,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
0,2014-01-01,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,1,2014,50.0,NaN
1,2014-02-01,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,2,2014,50.0,NaN
2,2014-03-01,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,3,2014,50.0,NaN
3,2014-04-01,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,4,2014,50.0,NaN
4,2014-05-01,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,5,2014,50.0,NaN


In [4]:
# retrieve unique currencies from database
symbols = list(df['cur_name'].unique())

#Add new versions of depricated currencies
symbols.extend(['ILS', 'BYN', 'MRU', 'SLE', 'VEF'])

symbols.sort()
symbolsString = ', '.join(symbols)
symbols

['AFN',
 'AMD',
 'AOA',
 'ARS',
 'AZN',
 'BDT',
 'BIF',
 'BOB',
 'BTN',
 'BYN',
 'BYR',
 'CDF',
 'CNY',
 'COP',
 'CVE',
 'DJF',
 'DOP',
 'DZD',
 'EGP',
 'ERN',
 'ETB',
 'GEL',
 'GHS',
 'GMD',
 'GNF',
 'GTQ',
 'HNL',
 'HTG',
 'IDR',
 'ILS',
 'INR',
 'IQD',
 'IRR',
 'JOD',
 'JPY',
 'KES',
 'KGS',
 'KHR',
 'KZT',
 'LAK',
 'LBP',
 'LKR',
 'LRD',
 'LSL',
 'LYD',
 'MDL',
 'MGA',
 'MMK',
 'MNT',
 'MRO',
 'MRU',
 'MWK',
 'MXN',
 'MZN',
 'NAD',
 'NGN',
 'NIO',
 'NIS',
 'NPR',
 'PAB',
 'PEN',
 'PHP',
 'PKR',
 'PYG',
 'RUB',
 'RWF',
 'SDG',
 'SLE',
 'SLL',
 'SOS',
 'SSP',
 'SYP',
 'SZL',
 'THB',
 'TJS',
 'TRY',
 'TZS',
 'UAH',
 'UGX',
 'USD',
 'VEF',
 'VEF',
 'VND',
 'XAF',
 'XOF',
 'YER',
 'ZAR',
 'ZMW']

In [5]:
# retrieve all unique dates in or after 1999 from database
uniqueDates = list(df['Date'].unique())
uniqueDates
past_1999Dates = list(filter(lambda x: x.year >= 1999, uniqueDates))
past_1999Dates
min(past_1999Dates)

AttributeError: 'numpy.datetime64' object has no attribute 'year'

In [ ]:
#testing format of converting date to string
tempDate = min(past_1999Dates)
tempDate.strftime('%Y-%m-%d')

'1999-01-01'

In [ ]:
# setting up API calling parameters
url = 'http://api.exchangeratesapi.io/v1/'
key = ''
base = 'USD'

responsesDict = dict() 

for date in uniqueDates:
    dateString = date.strftime('%Y-%m-%d')
    requestString ="{url}/{date}?access_key={api_key}&base=USD&symbols={symbols}".format(url=url,date=date,api_key=key,symbols=symbolsString)
    responseJson = requests.get(requestString).json()
    responsesDict[date.strftime('%Y-%m-%d')]=responseJson

with open('exchange_rates.json', 'w') as f:
    json.dump(responsesDict, f)

## Generate a JSON where the keys are currencies and the values are timestamps

In [6]:
with open('exchange_rates.json','r') as f:
    all_rates = json.load(f)

currency_timestamps = {}

for date, values in all_rates.items():
    try:
        rates = values['rates']

        for currency, rate in rates.items():
            if currency not in currency_timestamps:
                currency_timestamps[currency] = []
            currency_timestamps[currency].append(date)
    except:
        # print("No 'rates' for " + date)
        # print(values)
        continue

    

with open('currency_timestamps.json', 'w') as f:
    json.dump(currency_timestamps, f)

## Example of finding the closest date for a given currency using this new JSON

In [7]:

with open('currency_timestamps.json', 'r') as f:
    all_currency_timestamps = json.load(f)

currecny = 'AFN'
queried_date = dt.fromisoformat("2014-01-13")

date_list = all_currency_timestamps[currency]
closest_date = min(date_list, key=lambda list_element: abs(dt.fromisoformat(list_element) - queried_date))

print("Queried Date: " + queried_date.strftime("%Y-%m-%d") + "\nClosest Date: " + closest_date)



Queried Date: 2014-01-13
Closest Date: 2014-01-01
